# Accessing Yelp API

Kristan Bryan Simbulan

## Initialization

In [1]:
# Install Yelp API
!pip install yelpapi

# Install tmdbsimple (only need to run once)
!pip install tqdm

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
# Load API credentials
with open('/Users/kristansimbulan/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate Yelp API variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

## Accessing data from Yelp

In [6]:
# Intitiate 'location' and 'term' variables containing search keywords for location and term, respectively
LOCATION = "New York"
TERM = "Pasta"

In [7]:
## Specify JSON file for saving data
JSON_FILE = r"/Users/kristansimbulan/Documents/GitHub/Working-with-Yelp-API/yelp-search-new-york-pasta.json"
print(f'data will be saved to: {JSON_FILE}')

data will be saved to: /Users/kristansimbulan/Documents/GitHub/Working-with-Yelp-API/yelp-search-new-york-pasta.json


In [8]:
## Create a new empty json file (delete the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] /Users/kristansimbulan/Documents/GitHub/Working-with-Yelp-API/yelp-search-new-york-pasta.json already exists. Deleting previous file...
[i] /Users/kristansimbulan/Documents/GitHub/Working-with-Yelp-API/yelp-search-new-york-pasta.json not found. Saving empty list to new file.
- 0 previous results found.


530

In [9]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/530 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [10]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,5306,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,3186.380498
1,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3287,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.3,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
2,OCTiJwvjoK81WoDwsTkFvA,paesano-of-mulberry-street-new-york,Paesano of Mulberry Street,https://s3-media2.fl.yelpcdn.com/bphoto/yN9ARC...,False,https://www.yelp.com/biz/paesano-of-mulberry-s...,1496,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.1,"{'latitude': 40.718554, 'longitude': -73.997447}","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '136 Mulberry St', 'address2': ''...",+12129651188,(212) 965-1188,1481.706653
3,HodL0fXoF08C3Ir-kLeTtw,forma-pasta-factory-brooklyn,Forma Pasta Factory,https://s3-media2.fl.yelpcdn.com/bphoto/bFutbq...,False,https://www.yelp.com/biz/forma-pasta-factory-b...,707,"[{'alias': 'pastashops', 'title': 'Pasta Shops...",4.7,"{'latitude': 40.72371, 'longitude': -73.95157}","[delivery, pickup]",$$,"{'address1': '14 Bedford Ave', 'address2': Non...",+17183492009,(718) 349-2009,4141.117802
4,yXMZSuLJEpNu24yhKey9XA,forma-pasta-factory-brooklyn-3,Forma Pasta Factory,https://s3-media1.fl.yelpcdn.com/bphoto/6LEtDQ...,False,https://www.yelp.com/biz/forma-pasta-factory-b...,172,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.7,"{'latitude': 40.686023409875474, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '5 Greene Ave', 'address2': None,...",+13477991333,(347) 799-1333,2800.607052


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,vpkTNjq9dRh9rT3Nh0pe-A,nolita-pizza-new-york-2,Nolita Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/_BKVVo...,False,https://www.yelp.com/biz/nolita-pizza-new-york...,247,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.6,"{'latitude': 40.7208883, 'longitude': -73.9962...","[pickup, delivery]",$,"{'address1': '68 Kenmare St', 'address2': None...",+16468959131,(646) 895-9131,1725.054309
996,t-St6BqNMgecsvICeGgreA,joes-of-avenue-u-brooklyn,Joe's of Avenue U,https://s3-media3.fl.yelpcdn.com/bphoto/tfaTVI...,False,https://www.yelp.com/biz/joes-of-avenue-u-broo...,228,"[{'alias': 'sicilian', 'title': 'Sicilian'}, {...",3.9,"{'latitude': 40.597072, 'longitude': -73.97367}",[delivery],$$,"{'address1': '287 Avenue U', 'address2': '', '...",+17184499285,(718) 449-9285,12175.446917
997,auKeIP4_nJuWkBIUcDObvA,wahizza-ridgefield-park,Wahizza,https://s3-media4.fl.yelpcdn.com/bphoto/YUF7D1...,False,https://www.yelp.com/biz/wahizza-ridgefield-pa...,37,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.6,"{'latitude': 40.857423251606264, 'longitude': ...","[pickup, delivery]",NaN,"{'address1': '213 Main St', 'address2': None, ...",+12018706059,(201) 870-6059,17092.747276
998,L0XYugkXpuACPMQ451MB8g,el-mate-brooklyn,El Mate,https://s3-media3.fl.yelpcdn.com/bphoto/1Pq7B-...,False,https://www.yelp.com/biz/el-mate-brooklyn?adju...,361,"[{'alias': 'argentine', 'title': 'Argentine'}]",4.6,"{'latitude': 40.6749782203506, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '550 Court St', 'address2': '', '...",+17182221102,(718) 222-1102,3416.302173
999,W0sXHSSpkiMEDJiBmSLNYQ,trestle-astoria,Trestle,https://s3-media2.fl.yelpcdn.com/bphoto/7xXAAf...,False,https://www.yelp.com/biz/trestle-astoria?adjus...,564,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,"{'latitude': 40.7606154, 'longitude': -73.9229...","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '34-02 Broadway', 'address2': '',...",+13478080290,(347) 808-0290,8588.574048


In [11]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

30

In [12]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [14]:
# save the final results to a compressed csv
final_df.to_csv(r"/Users/kristansimbulan/Documents/GitHub/Working-with-Yelp-API/final_results_NY_pizza.csv.gz", compression='gzip',index=False)